In [ ]:
import getpass
import os
#使用langSmith追踪过程, 注意langSmith只有免费5k次调用/月
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = 'xxx'
os.environ['ZHIPUAI_API_KEY'] = 'xxx'
from langchain_community.chat_models import ChatZhipuAI
from langchain_community.embeddings import ZhipuAIEmbeddings
import logging

# 设置日志级别
logging.basicConfig(level=logging.INFO)
# 问题：如何评测embedding模型的性能？
gen_model = ChatZhipuAI(
    model="glm-4-plus",
)
embedding_model = ZhipuAIEmbeddings(
    model = 'embedding-3',
    dimensions=2048
)
import bs4
from langchain import hub
from langchain_community.document_loaders.base import BaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_community.document_loaders.parsers import GrobidParser
from langchain_community.document_loaders.generic import GenericLoader

## Load PDF

In [2]:
pdf_dir = '/Users/liyihang/Documents/大模型+微调'

loader = GenericLoader.from_filesystem(
            pdf_dir,
            glob="*",
            suffixes=[".pdf"],
            parser= GrobidParser(segment_sentences=False)
        )
docs = loader.load()
filter_docs = [doc for doc in docs if len(doc.page_content.split(' '))<30]

/Users/liyihang/miniconda3/envs/llm/lib/python3.9/site-packages/langchain_community/document_loaders/parsers/grobid.py:53: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(xml_data, "lxml")


In [3]:
# 展示一下docs的内容
print("metadata包括:", docs[0].metadata.keys())
print("文档所属论文:",docs[0].metadata['paper_title'])
print("文档所属章节:",docs[0].metadata['section_title'])
print(f"文档所属章节编号:{docs[0].metadata['section_number']}, 所属论文页码:{docs[0].metadata['pages']}, 文件路径:{docs[0].metadata['file_path']}")
print(f"发表时间:{docs[0].metadata['pub_time']}")

metadata包括: dict_keys(['text', 'para', 'bboxes', 'pages', 'section_title', 'section_number', 'paper_title', 'file_path', 'pub_time'])
文档所属论文: COIG-CQIA: Quality is All You Need for Chinese Instruction Fine-tuning
文档所属章节: Abstract
文档所属章节编号:0, 所属论文页码:(1, 1), 文件路径:/Users/liyihang/Documents/大模型+微调/COIG-COIA.pdf
发表时间:N/A


In [4]:
# 提取docs metadata中的abstract， 作为一级摘要文档，方便查询，metadata为paper_title, pub_time
prev_paper_title = ''
abstract_docs = []
for doc in docs:
    if doc.metadata['paper_title'] != prev_paper_title:
        #新的文章，提取abstract
        prev_paper_title = doc.metadata['paper_title']
        asb_doc = Document(
            page_content=prev_paper_title + '\n' + doc.page_content,  # 文本内容
            metadata={"paper_title": prev_paper_title, "pub_time":doc.metadata['pub_time']}  # 元数据
        )
        abstract_docs.append(asb_doc)
    else:
        continue  

## Split Docs


In [5]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(encoding_name='cl100k_base', chunk_size=512, chunk_overlap=32)
all_splits = text_splitter.split_documents(filter_docs)
def split_array(arr:list[Document], single_array_size:int)->list[list[Document]]:
    # 使用列表切片按 single_array_size 拆分list
    # 因为zhipuAI embedding在调用add_documents时最大允许的文档数是64，所以要分批add
    return [arr[i:i + single_array_size] for i in range(0, len(arr), single_array_size)]
split_docs = split_array(all_splits, 64)
abstract_split_docs = split_array(abstract_docs, 64)

## Store Vectors


In [6]:
import hashlib
from langchain_community.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever
def calculate_md5_from_filepaths(folder_path):
    """
    根据文件夹下所有文件的相对路径计算 MD5 码
    :param folder_path: 文件夹路径
    :return: MD5 码（字符串）
    """
    filepaths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # 获取文件的相对路径
            relative_path = os.path.relpath(os.path.join(root, file), folder_path)
            filepaths.append(relative_path)
    
    # 按字母顺序排序文件路径
    filepaths.sort()
    
    # 将所有文件路径拼接成一个字符串
    filepaths_str = ''.join(filepaths)
    
    # 计算 MD5 值
    md5_hash = hashlib.md5(filepaths_str.encode('utf-8')).hexdigest()
    
    return md5_hash
md5_value = calculate_md5_from_filepaths(pdf_dir)
def get_faiss_index(splits_docs, md5_value, index_path, embedding_model, index_name='faiss_index'):
    FAISS_INDEX_PATH = f"{index_path}/{md5_value}_{index_name}"
    #print(f"开始创建向量数据库：{FAISS_INDEX_PATH}")
    if os.path.exists(FAISS_INDEX_PATH):
        # 加载已存在的向量数据库
        vectordb = FAISS.load_local(
            FAISS_INDEX_PATH,
            embedding_model,
            allow_dangerous_deserialization=True  # 明确允许反序列化
        )
        print(f"已加载本地向量数据库{index_name}")
    else:
        #index
        for i, docs in enumerate(splits_docs):
            if i == 0:
                vectordb = FAISS.from_documents(documents=docs, embedding=embedding_model)
            else:
                vectordb.add_documents(documents=docs)
        vectordb.save_local(FAISS_INDEX_PATH)
    return vectordb

content_vectordb = get_faiss_index(split_docs,md5_value=md5_value, index_path='/Users/liyihang/code/langchain_study/rag_agent_simple/faiss_db', embedding_model=embedding_model)
abstract_vectordb = get_faiss_index(abstract_split_docs,md5_value=md5_value, index_path='/Users/liyihang/code/langchain_study/rag_agent_simple/faiss_db', embedding_model=embedding_model, index_name='abstract_faiss_index')

INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


已加载本地向量数据库faiss_index
已加载本地向量数据库abstract_faiss_index


## Retrieve & Generate

In [7]:
import os
from langgraph.graph import START, MessagesState, StateGraph, END
from typing import Annotated, TypedDict, Sequence
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage, ToolMessage
from prompts import judge_prompt, rag_prompt, direct_prompt, choose_tool_prompt
from langchain_community.document_transformers import LongContextReorder
from functools import partial
from langgraph.prebuilt import ToolNode, tools_condition
from custom_tools.multi_query import *
from custom_tools.rag_fusion import *
from custom_tools.decomposition import *
from langchain_core.tools import Tool, InjectedToolArg, tool
from langchain_core.runnables import RunnableLambda, Runnable
from langchain_core.vectorstores import VectorStoreRetriever
from typing import Callable

retriever_store = {}
def gen_title_filter(candidate_titles:Sequence[str]):
    return {"paper_title":{"$in":candidate_titles}}


    

@tool(parse_docstring=True)
def multi_query_tool(question:str, retriever_id: Annotated[str, InjectedToolArg]):
    '''为了缓解余弦相似度查询的局限性，生成多个语义类似但措辞不同的查询，以获取更全面的上下文

    Args:
        question (str): 用户提出的问题
        retriever_id (Annotated[str, InjectedToolArg]): 查询工具
    '''
    # 从全局字典中获取 retriever
    retriever = retriever_store.get(retriever_id)
    if retriever is None:
        raise ValueError(f"Invalid retriever_id: {retriever_id}")
    query_search_func = create_multi_query_chain(retriever)
    if isinstance(query_search_func, Runnable):
        return create_multi_query_chain(retriever).invoke({"question":question})
    elif isinstance(query_search_func, Callable):
        return query_search_func(question)
    else:
        raise NotImplementedError

@tool(parse_docstring=True)
def rag_fusion_query_tool(question:str, retriever_id: Annotated[str, InjectedToolArg]):
    '''生成关于用户问题的**多个方面**的查询，以提供对原始问题的**不同角度**的检索结果，把这些结果和原始查询一起进行倒数排序融合，最终返回查询到的Document。

    Args:
        question (str): 用户提出的问题
        retriever_id (Annotated[str, InjectedToolArg]): 查询工具
    '''
    # 从全局字典中获取 retriever
    retriever = retriever_store.get(retriever_id)
    if retriever is None:
        raise ValueError(f"Invalid retriever_id: {retriever_id}")
    query_search_func = create_abstract_query_chain(retriever)
    if isinstance(query_search_func, Runnable):
        return create_abstract_query_chain(retriever).invoke({"question":question})
    elif isinstance(query_search_func, Callable):
        return query_search_func(question)
    else:
        raise NotImplementedError

@tool(parse_docstring=True)
def decomposition_query_tool(question:str, retriever_id: Annotated[str, InjectedToolArg]):
    '''对用户提出的问题进行分解，生成多个子问题，然后分别查询，最后将查询结果进行融合。

    Args:
        question (str): 用户提出的问题
        retriever_id (Annotated[str, InjectedToolArg]): 查询工具
    '''
    # 从全局字典中获取 retriever
    retriever = retriever_store.get(retriever_id)
    if retriever is None:
        raise ValueError(f"Invalid retriever_id: {retriever_id}")
    query_search_func = create_decomposition_search(retriever)
    if isinstance(query_search_func, Runnable):
        return create_decomposition_search(retriever).invoke({"question":question})
    elif isinstance(query_search_func, Callable):
        return query_search_func(question)
    else:
        raise NotImplementedError

@tool(parse_docstring=True)
def create_abstract_query_tool(question:str, retriever: Annotated[VectorStoreRetriever, InjectedToolArg]):
    '''生成关于用户问题的**多个方面**的查询，以提供对原始问题的**不同角度**的检索结果，把这些结果和原始查询一起进行倒数排序融合，最终返回查询到的Document。

    Args:
        question (str): 用户提出的问题
        retriever (Annotated[VectorStoreRetriever, InjectedToolArg]): 查询工具
    '''
    query_search_func = create_abstract_query_chain(retriever)
    if isinstance(query_search_func, Runnable):
        return create_abstract_query_chain(retriever).invoke({"question":question})
    elif isinstance(query_search_func, Callable):
        return query_search_func(question)
    else:
        raise NotImplementedError

In [ ]:
from custom_tools.common import docs2str, fill_messages
from copy import deepcopy
from uuid import uuid4
from langchain_core.runnables import chain
# ===== 状态定义 =====
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    needs_rag: bool  # 标记是否需要RAG
    query: str      # 存储检索内容
    cur_node: str     # 记录当前节点名称
    title_list: list[str] # 和当前问题相关的论文标题列表
    search_type: str # 检索类型
    search_kwargs: dict # 检索参数
# ===== llm定义 =====
judge_llm = ChatZhipuAI(model="glm-4-plus")
answer_llm = ChatZhipuAI(model="glm-4-plus")
tooluse_llm = ChatZhipuAI(model="glm-4-plus")
tools = [multi_query_tool, rag_fusion_query_tool, decomposition_query_tool]
tooluse_llm = tooluse_llm.bind_tools(tools)

def inject_retriever(ai_msg, vectordb, search_type:str, search_kwargs:dict):
    tool_calls = []
    for tool_call in ai_msg.tool_calls:
        tool_call_copy = deepcopy(tool_call)
        # 生成唯一的 retriever_id
        retriever_id = str(uuid4())
        # 创建 retriever 并存储
        retriever = vectordb.as_retriever(search_type=search_type, search_kwargs=search_kwargs)
        retriever_store[retriever_id] = retriever
        # 将 retriever_id 注入参数
        tool_call_copy["args"]["retriever_id"] = retriever_id
        tool_calls.append(tool_call_copy)
    ai_msg.tool_calls = tool_calls
    return ai_msg

# ===== 节点函数 =====
def judge_node(state: State):
    """判断是否需要RAG"""
    # 构造判断提示
    prompt = judge_prompt.format_messages(messages=state["messages"])
    response = judge_llm.invoke(prompt).content
    #state["needs_rag"] = True if "Y" in response.upper() else False
    need_rag = True if "Y" in response.upper() else False
    return {"needs_rag": need_rag, "cur_node":"judge_node"}

def locate_paper(state:State):
    abstract_retreiver = abstract_vectordb.as_retriever(search_type = state.get("search_type", "similarity_score_threshold"), search_kwargs = state.get("search_kwargs", {'k':5, 'score_threshold':0.3}))
    abstract_docs = create_abstract_query_tool.invoke({"question":"deepseek论文中都有哪些创新点?", "retriever":abstract_retreiver})
    paper_list = [doc[0].metadata['paper_title'] for doc in abstract_docs]
    print("检索到相关论文：", paper_list)
    return {"title_list":paper_list, "cur_node":"locate_paper"}

def select_retrieve_node(state: State):
    """模拟检索上下文（实际应替换为真实检索逻辑）"""
    print("\n=== 深入检索pdf... ===")
    prompt = choose_tool_prompt.format_messages(question=state["messages"][-1].content)
    response = tooluse_llm.invoke(prompt)
    return {'messages':[response], "query":state["messages"][-1].content, "cur_node":"select_retrieve_node"}

def inject_tools(state: State):
    """注入工具"""
    title_filter = gen_title_filter(state["title_list"])
    tool_msg = inject_retriever(ai_msg=state["messages"][-1], vectordb=content_vectordb, search_type=state.get("search_type", "similarity"), search_kwargs=state.get("search_kwargs", {'k':5, 'fetch_k':50, 'filter':title_filter}))
    return {'messages':[tool_msg], "cur_node":"inject_tools"}

def answer_with_rag(state: State):
    """使用RAG生成回答"""
    prompt = rag_prompt.format_messages(
        context=state["messages"][-1].content,
        messages=fill_messages(content=state["query"], role="user")
    )
    response = answer_llm.invoke(prompt)
    return {"messages": [AIMessage(response.content)], "cur_node":"answer_with_rag"}

def answer_directly(state: State):
    """直接生成回答"""
    prompt = direct_prompt.format_messages(messages=state["messages"])
    response = answer_llm.invoke(prompt)
    return {"messages": [AIMessage(response.content)], "cur_node":"answer_directly"}

def output_node(state: State):
    return {"cur_node":"end"}

retrieve_tool_node = ToolNode(tools=tools)


In [ ]:
from langgraph.checkpoint.memory import MemorySaver
# ===== 构建工作流 =====
workflow = StateGraph(State)

# 添加节点
workflow.add_node("check_exit", judge_node)
workflow.add_node("locate_paper", locate_paper)
workflow.add_node("select_retrieve", select_retrieve_node)
workflow.add_node("tools", retrieve_tool_node)
workflow.add_node("answer_rag", answer_with_rag)
workflow.add_node("answer_direct", answer_directly)
workflow.add_node("output", output_node)
workflow.add_node("inject_tools", inject_tools)
# 设置边
workflow.set_entry_point("check_exit")

# 条件分支
workflow.add_conditional_edges(
    "check_exit",
    lambda state: END if "再见！" in str(state["messages"][-1]) else ("locate_paper" if state["needs_rag"] else "answer_direct")
)
workflow.add_edge("locate_paper", "select_retrieve")
workflow.add_edge("select_retrieve", "inject_tools")
workflow.add_conditional_edges("inject_tools", tools_condition)
workflow.add_edge("tools", "answer_rag")
workflow.add_edge("answer_rag", "output")
workflow.add_edge("answer_direct", "output")
workflow.add_edge("output", END)

# 持久化记忆
memory = MemorySaver()


# ===== 对话循环 =====
def chat_loop(app):
    print("论文助手：您好！我是学术论文阅读助手，请输入您的问题（输入'退出'或'exit'结束）")
    
    thread_id = "user_123"  # 用户唯一标识，用于保存和恢复对话内容
    while True:
        user_input = input("\n用户：")
        
        # 初始化或加载对话状态
        config = {"configurable": {"thread_id": thread_id}}
        # 调用工作流
        for event in app.stream(
            {"messages": [HumanMessage(user_input)]},
            config=config,
            stream_mode="values"
        ):
            if "cur_node" in event.keys() and (event["cur_node"] == "end"):
                event["messages"][-1].pretty_print()
                
app = workflow.compile(checkpointer=memory)
from IPython.display import Image, display
from PIL import Image as PILImage
from io import BytesIO

def gen_graph_img(app):
    try:
        # 获取图片的二进制数据
        png_data = app.get_graph().draw_mermaid_png()
        
        # 将二进制数据转换为PIL图像对象
        image = PILImage.open(BytesIO(png_data))
        
        # 保存图像到指定路径
        image.save('graph.png')  # 替换为你想保存的路径

    except Exception as e:
        # 处理可能出现的错误（例如缺少依赖）
        print(f"An error occurred: {e}")

gen_graph_img(app)

In [10]:
# 初始化或加载对话状态
thread_id = "user123"
config = {"configurable": {"thread_id": thread_id}}
for event in app.stream(
    {"messages": [HumanMessage("deepseek论文中都有哪些创新点?")]},
    config=config,
    stream_mode="values"
):
    #if "cur_node" in event.keys() and (event["cur_node"] == "end" or event["cur_node"] == "inject_tools"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

deepseek论文中都有哪些创新点?


INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


================================ Human Message =================================

deepseek论文中都有哪些创新点?


INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/embeddings "HTTP/1.1 200 OK"



=== 生成的查询 ===
1. **用户初始查询**: deepseek论文中都有哪些创新点?
2. **查询1**: DeepSeek论文的主要技术创新和应用突破
3. **查询2**: DeepSeek研究论文提出的独特算法和模型改进
4. **查询3**: DeepSeek论文在相关领域的创新贡献和影响
5. **查询4**: DeepSeek论文中提到的实验方法和结果创新点


...

================================== Ai Message ==================================
Tool Calls:
  decomposition_query_tool (call_-8891222663972289200)
 Call ID: call_-8891222663972289200
  Args:
    question: deepseek论文中都有哪些创新点?
================================== Ai Message ==================================
Tool Calls:
  decomposition_query_tool (call_-8891222663972289200)
 Call ID: call_-8891222663972289200
  Args:
    question: deepseek论文中都有哪些创新点?
    retriever_id: eef48cae-d3c4-4085-b46d-2b68d0c2d40b


INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


================================= Tool Message =================================
Name: decomposition_query_tool

根据提供的文档信息，DeepSeek论文（具体为"DeepSeekMath: Pushing the Limits of Mathematical Reasoning in Open Language Models"）中的技术和方法相比现有研究具有以下具体的创新点：

### 1. 高质量的DeepSeekMath Corpus数据集
- **多语言覆盖**：DeepSeekMath Corpus涵盖了多语言的数学内容，相比现有研究中的数据集，具有更广泛的适用性和普适性。
- **大规模数据**：该数据集在规模上是最大的，提供了丰富的数学推理任务数据，为模型的训练和优化提供了坚实的基础。
- **高质量验证**：通过预训练实验和数据分析，验证了DeepSeekMath Corpus的高质量，确保其在数学推理任务中的有效性和可靠性。

### 2. 系统的数据分析和验证方法
- **对比分析**：通过预训练实验，对比了DeepSeekMath Corpus与其他最近发布的数学训练数据集，验证其质量和效果，这种方法在现有研究中较少见。
- **质量验证**：通过实验和数据分析，系统地验证了数据集的质量和覆盖范围，确保其在实际应用中的有效性。

### 3. 模型结构和算法的优化
- **结构优化**：探索和优化模型的结构，使其在数学推理任务中表现更佳，这一点在现有研究中往往被忽视。
- **算法改进**：改进算法以提高模型的数学推理能力，特别是在处理复杂数学问题时，显示出更高的准确性和效率。

### 4. 创新的技术手段
- **梯度分区技术**：虽然这一点来自其他文档，但可能与DeepSeek论文的技术有相似之处。使用梯度分区技术在训练大规模模型时，减少内存消耗，同时保持通信量不变，这在现有研究中是一个创新点。

### 总结
DeepSeek论文通过构建高质量的多语言数学数据集、系统地进行数据分析和验证、优化模型结构和算法，以及采用创新的技术手段，显著提升了开放语言模型在数学推理方面的能力。这些创新点不仅填补了现有研究的空白，也为未来的数学推

INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

DeepSeek论文（具体指"DeepSeekMath: Pushing the Limits of Mathematical Reasoning in Open Language Models"）在数学推理领域的开放语言模型研究方面，提出了若干显著的创新点，具体如下：

### 1. 高质量的DeepSeekMath Corpus数据集
- **多语言覆盖性**：DeepSeekMath Corpus数据集涵盖了多种语言的数学内容，显著扩展了数据集的适用范围，相较于现有数据集，更具普适性和广泛性。
- **大规模性**：该数据集在规模上达到了前所未有的水平，提供了丰富的数学推理任务数据，为模型的深度训练和性能优化奠定了坚实基础。
- **质量验证**：通过系统的预训练实验和数据分析，验证了DeepSeekMath Corpus的高质量特性，确保其在数学推理任务中的有效性和可靠性。

### 2. 系统的数据分析和验证方法
- **对比分析**：通过预训练实验，对DeepSeekMath Corpus与其他近期发布的数学训练数据集进行了详细对比，验证了其质量和效果，此类方法在现有研究中较为罕见。
- **质量验证**：通过实验和数据分析，系统地验证了数据集的质量和覆盖范围，确保其在实际应用中的有效性和广泛适用性。

### 3. 模型结构和算法的优化
- **结构优化**：对模型结构进行了深入探索和优化，使其在数学推理任务中表现出更高的性能，弥补了现有研究中对模型结构优化的忽视。
- **算法改进**：对算法进行改进，显著提升了模型在处理复杂数学问题时的准确性和效率。

### 4. 创新的技术手段
- **梯度分区技术**：尽管此点源自其他文档，但可能与DeepSeek论文的技术思路有共通之处。采用梯度分区技术在训练大规模模型时，有效减少了内存消耗，同时保持通信量不变，这一技术在现有研究中具有显著的创新性。

### 总结
DeepSeek论文通过构建高质量的多语言数学数据集、系统地进行数据分析和验证、优化模型结构和算法，以及采用创新的技术手段，显著提升了开放语言模型在数学推理方面的能力。这些创新点不仅

In [19]:
chat_loop(app)

论文助手：您好！我是学术论文阅读助手，请输入您的问题（输入'退出'或'exit'结束）


INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"



=== 已有知识无法回答，正在检索pdf... ===


INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


[HumanMessage(content='deepseek论文的创新点是什么？', additional_kwargs={}, response_metadata={}, id='ea270908-79f9-4258-a042-748cbfd13867'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"__arg1": "deepseek论文的创新点是什么？"}', 'name': 'decompose_search'}, 'id': 'call_-8891208954433086323', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 492, 'total_tokens': 511}, 'model_name': 'glm-4-plus', 'finish_reason': 'tool_calls'}, id='run-ecc7f45e-4456-43b7-9330-702bc66c76df-0', tool_calls=[{'name': 'decompose_search', 'args': {'__arg1': 'deepseek论文的创新点是什么？'}, 'id': 'call_-8891208954433086323', 'type': 'tool_call'}]), ToolMessage(content='根据提供的文档信息和之前的问答对，我们可以总结出DeepSeek论文中提出的技术或方法相比现有研究的独特创新之处如下：\n\n### 1. **DeepSeekMath Corpus的高质量和多语言特性**\n   - **创新点**：\n     - **高质量**：DeepSeekMath语料库强调其高质量，这意味着其在数据准确性、完整性和适用性方面优于现有的数学训练语料库。\n     - **多语言支持**：涵盖多语言数学内容，填补了现有语料库在多语言支持方面的空白，使得模型能够更好地处理不同语言的数学问题。\n   - **对比现有

INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

DeepSeek论文在开放语言模型应用于数学推理领域展现了多项显著创新点，具体可概述如下：

### 1. **高质量与多语言特性的DeepSeekMath Corpus构建**
   - **创新阐述**：DeepSeekMath Corpus不仅强调数据的高质量，确保其在准确性、完整性和适用性方面超越现有语料库，而且实现了多语言覆盖，显著拓展了语料库的应用广度。
   - **对比分析**：相较于现有语料库普遍存在的质量不一及语言单一性问题，DeepSeekMath Corpus通过提升数据质量及多语言支持，有效解决了这些局限，为跨语言数学推理研究提供了坚实基础。

### 2. **系统全面的预训练实验与比较研究**
   - **创新阐述**：论文通过设计系统性的预训练实验，对DeepSeekMath语料库与其他先进数学训练语料库进行了详尽的性能比较，提供了充分的实验数据支持。
   - **对比分析**：现有研究往往缺乏此类系统比较，DeepSeek论文通过全面实验验证，增强了研究结果的可信度与说服力。

### 3. **多种强化学习方法的综合应用与深入分析**
   - **创新阐述**：论文综合运用了SFT、RFT、Online RFT、DPO、PPO和GRPO等多种强化学习方法，并对每种方法进行了细致的推导与分析，展示了研究方法的丰富性和理论深度。
   - **对比分析**：相较于多数研究仅聚焦于某几种方法，DeepSeek论文的多方法综合应用及深入剖析，提供了更为全面的优化策略。

### 4. **Direct Preference Optimization (DPO)的引入与应用**
   - **创新阐述**：通过引入DPO方法，直接针对模型偏好进行优化，有效提升了模型性能。
   - **对比分析**：现有研究较少涉及DPO或未深入探讨其应用，DeepSeek论文在此方面的探索，展现了方法创新与应用潜力。

### 5. **前瞻性未来研究方向的探讨**
   - **创新阐述**：论文不仅局限于当前研究成果，还明确指出了三个关键组件的未来研究方向，体现了研究的持续性与前瞻性